Get database session


In [ ]:
# Import the connection function from the snowexsql library
from snowexsql.db import get_db
from sqlalchemy import inspect
from snowexsql.conversions import query_to_geopandas, query_to_pandas
from snowexsql.db import get_table_attributes
from sqlalchemy.sql import func

import shapely.geometry
from geoalchemy2.shape import from_shape
import geoalchemy2.functions as gfunc
import matplotlib.pyplot as plt
import datetime

%config InlineBackend.figure_format='retina'

# This is what you will use for all of hackweek to access the db
db_name = 'snow:hackweek@db.snowexdata.org/snowex'

engine, session = get_db(db_name)

print('SnowEx Database successfully loaded!')

from snowexsql.data import PointData, LayerData, ImageData, SiteData

# tables available in database ['spatial_ref_sys', 'points', 'layers', 'sites', 'images']


GPR 1 = Dates Jan 28 - 2nd 2020

In [ ]:
# We're trying to count all ASO snow depth tiles within this bbox
bbox_WSEN = 742000, 4322000, 747000, 4325000 # EPSG 32612?
x1, y1, x2, y2 = bbox_WSEN
polygon = shapely.geometry.Polygon([[x1, y1], [x1, y2], [x2, y2], [x2, y1]]) # used box() before
wkb_element = from_shape(polygon, srid=26912) # which srid is right? #srid = 26912

# Retrieve some of the GPR data
dataset = 'two_way_travel'
query = session.query(PointData.geom, PointData.easting, PointData.northing, PointData.value, PointData.date)

query = query.filter(PointData.type == dataset)

query = query.filter(PointData.date <=(datetime.date(2020,2,2))) # or PointData.date <=( datetime.date(2020,2,6))) 
#query = query.filter(PointData.date == "2020-02-04")

# Filter the query by bounding box (the part that doesn't work)
query = query.filter(gfunc.ST_Within(PointData.geom,wkb_element))

result = query.count()
print(result)

In [ ]:
# data to geopanda data frame

GPR1 = query_to_geopandas(query,engine)

In [ ]:
# to visualize the table 
GPR1.head()


In [ ]:
#rename column  ( value to twt)
GPR1.rename(columns={'value':'twt'},inplace=True )


In [ ]:
#plot to see bounding box
# Get the Matplotlib Axes object from the dataframe object, color points by snow depth value
ax = GPR1.plot(column='twt', legend=True, cmap='PuBu')
# Use non-scientific notation for x and y ticks
ax.ticklabel_format(style='plain', useOffset=False)
# Set the various plots x/y labels and title.
ax.set_title('Grand Mesa Jan 28th - Feb 1' )
ax.set_xlabel('Easting [m]')
ax.set_ylabel('Northing [m]')


In [ ]:
#Export as CSV
#will export to location of terminal
GPR1.to_csv('GPR1.csv')

GPR 2 = Dates February 4 - 9th 2020

In [ ]:
# We're trying to count all ASO snow depth tiles within this bbox
bbox_WSEN = 742000, 4322000, 747000, 4325000 # EPSG 32612?
x1, y1, x2, y2 = bbox_WSEN
polygon = shapely.geometry.Polygon([[x1, y1], [x1, y2], [x2, y2], [x2, y1]]) # used box() before
wkb_element = from_shape(polygon, srid=26912) # which srid is right? #srid = 26912

# Retrieve some of the GPR data
dataset = 'two_way_travel'
query = session.query(PointData.geom, PointData.easting, PointData.northing, PointData.value, PointData.date)

query = query.filter(PointData.type == dataset)

query = query.filter(PointData.date >=(datetime.date(2020,2,4))) # or PointData.date <=( datetime.date(2020,2,6))) 
#query = query.filter(PointData.date == "2020-02-04")

# Filter the query by bounding box (the part that doesn't work)
query = query.filter(gfunc.ST_Within(PointData.geom,wkb_element))

result = query.count()
print(result)

#result = session.query(PointData.date).filter(PointData.instrument == 'two_way_time').distinct().all()
#print(result)

In [ ]:
# data to geopanda data frame

GPR2 = query_to_geopandas(query,engine)

#rename column  ( value to twt)
GPR2.rename(columns={'value':'twt'},inplace=True )

#plot to see bounding box
# Get the Matplotlib Axes object from the dataframe object, color points by snow depth value
ax = GPR2.plot(column='twt', legend=True, cmap='PuBu')
# Use non-scientific notation for x and y ticks
ax.ticklabel_format(style='plain', useOffset=False)
# Set the various plots x/y labels and title.
ax.set_title('Grand Mesa Feb. 4 - 9th 2020' )
ax.set_xlabel('Easting [m]')
ax.set_ylabel('Northing [m]')


In [ ]:
GPR2.head()


In [ ]:
#export GPR2 
GPR2.to_csv('GPR2.csv')


In [ ]:
session.close()